In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [6]:
url = "https://raw.githubusercontent.com/maxxxxc/SIR-Summer-2023/main/dataset/magic04.data"
column_names = ["X1", "X2", "X3", "X4", "X5", "X6", "X7", "X8", "X9", "X10", "Y"]
df = pd.read_csv(url, header=None, names=column_names)

X = df.drop("Y", axis=1)
y = df["Y"]
print(y.value_counts())
y = y.replace({'g': 1, 'h': 0})
print(y.value_counts())

g    12332
h     6688
Name: Y, dtype: int64
1    12332
0     6688
Name: Y, dtype: int64


In [7]:
#divide

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
#param_grid = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}
#svm = SVC(kernel='rbf')

#grid_search = GridSearchCV(svm, param_grid, cv=5)
#grid_search.fit(X_train_scaled, y_train)

#best_params = grid_search.best_params_
#best_C = best_params['C']
#best_gamma = best_params['gamma']

best_C = 1
best_gamma = 1

#Fit the SVM with the best parameters:
svm = SVC(kernel='rbf', C=best_C, gamma=best_gamma)

In [11]:
svm.fit(X_train_scaled, y_train)

SVC(C=1, gamma=1)

In [12]:
# Make predictions on the test set
y_test_pred = svm.predict(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", accuracy)

# Prediction rule is essentially: 1 if distance > 0
distances = svm.decision_function(X_test_scaled)
tmp = np.where(distances > 0, 1, 0)
print("Match between Prediction and Rule based on Distance")
accuracy_score(tmp, y_test_pred)

Test Accuracy: 0.8622502628811777
Match between Prediction and Rule based on Distance


1.0

In [13]:
from sklearn.calibration import CalibratedClassifierCV

In [14]:
svm_platt = CalibratedClassifierCV(svm)
svm_platt.fit(X_train_scaled, y_train)
y_test_prob = svm_platt.predict_proba(X_test_scaled)

In [15]:
y_test_pred = np.where(y_test_prob[:,1] > 0.5, 1, 0)
accuracy_score(y_test, y_test_pred)

0.8642218717139852

In [44]:
def svm_iterate_process(X, y):
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    
    # Define the number of batches
    num_batches = 11
    
    # Randomly shuffle the data indices
    indices = np.random.permutation(len(X_train))
    
    # Calculate the batch size
    batch_size = len(X_train) // num_batches
    
    # Make predictions on the test set using majority voting
    preds_voting = np.zeros(len(y_test))
    # Make predictions on the test set using average of logit
    preds_distance = np.zeros(len(y_test))
    #Make predictions on the test set using average of probs
    preds_prob = np.zeros(len(y_test))

    
    # Split the training data into batches, fit a logistic regression model on each batch
    for i in range(num_batches):
        # Calculate the starting and ending indices for the current batch
        start_index = i * batch_size
        end_index = (i + 1) * batch_size
        
        # Select the current batch for training
        X_batch = X_train.iloc[indices[start_index:end_index]]
        y_batch = y_train.iloc[indices[start_index:end_index]]
        
        scaler = StandardScaler()
        X_batch_scaled = scaler.fit_transform(X_batch)
        X_test_scaled = scaler.transform(X_test)
        
        
        # Create a support vector machine model
        param_grid = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}
        svm = SVC(kernel='rbf')

        grid_search = GridSearchCV(svm, param_grid, cv=5)
        grid_search.fit(X_batch_scaled, y_batch)

        best_params = grid_search.best_params_
        best_C = best_params['C']
        best_gamma = best_params['gamma']
        svm = SVC(kernel='rbf', C=best_C, gamma=best_gamma)
        
        # Fit the model on the current batch
        svm.fit(X_batch_scaled, y_batch)
        
               
        # Accumulate the predictions using majority voting
        y_pred = svm.predict(X_test_scaled)
        preds_voting += (y_pred == 1)

    
        # Accumulate the predictions using majority voting
        y_pred = svm.decision_function(X_test_scaled)
        preds_distance += y_pred
        
        
        #Accumulate the probs
        svm_platt = CalibratedClassifierCV(svm)
        svm_platt.fit(X_batch_scaled, y_batch)
        y_pred = svm_platt.predict_proba(X_test_scaled)
        preds_prob += y_pred[:,1]
   
    
    accuracy = np.zeros(4)
    
    # Majority voting (selecting the most frequent prediction for each sample)
    final_predictions = np.where(preds_voting > num_batches / 2, 1, 0)
    accuracy[0] = accuracy_score(y_test, final_predictions)
    # Average of logit
    final_predictions = np.where(preds_distance > 0, 1, 0)
    accuracy[1] = accuracy_score(y_test, final_predictions)
    #Average of probs
    final_predictions = np.where(preds_prob / num_batches > 0.5, 1, 0)
    accuracy[2] = accuracy_score(y_test, final_predictions)
    
    # Train a model on all 11 batches of training data
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    param_grid = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}
    svm = SVC(kernel='rbf')

    grid_search = GridSearchCV(svm, param_grid, cv=5)
    grid_search.fit(X_train_scaled, y_train)

    best_params = grid_search.best_params_
    best_C = best_params['C']
    best_gamma = best_params['gamma']
    svm = SVC(kernel='rbf', C=best_C, gamma=best_gamma)
    
    svm.fit(X_train_scaled, y_train)
    y_pred = svm.predict(X_test_scaled)
    accuracy[3] = accuracy_score(y_test, y_pred)
    
    return accuracy

In [45]:
# Number of times to repeat the process
num_repeats = 50

# Initialize an empty matrix (10-by-4) to store accuracies
accuracies = np.zeros((num_repeats, 4))

seed = 42
# Repeat the process and store accuracies
for i in range(num_repeats):
    np.random.seed(seed)
    accuracies[i] = svm_iterate_process(X, y)
    seed += 2
    

# Print the accuracies
print("Accuracies:", accuracies)

Accuracies: [[0.86303891 0.86461619 0.86579916 0.87276551]
 [0.86303891 0.86422187 0.86711356 0.87210831]
 [0.87315983 0.87407992 0.87486856 0.88025762]
 [0.86501052 0.86632492 0.86750789 0.8701367 ]
 [0.86737645 0.86882229 0.86921661 0.87237119]
 [0.86225026 0.86369611 0.86409043 0.86895373]
 [0.86461619 0.86461619 0.86501052 0.87210831]
 [0.86317035 0.86750789 0.86514196 0.87868034]
 [0.85962145 0.8623817  0.86251314 0.86855941]
 [0.86448475 0.86606204 0.86343323 0.87118822]
 [0.86816509 0.86671924 0.86685068 0.8694795 ]
 [0.86632492 0.86685068 0.86724501 0.87315983]
 [0.85554679 0.85830705 0.85922713 0.86619348]
 [0.86356467 0.86356467 0.86435331 0.86987382]
 [0.86211882 0.86435331 0.86356467 0.87197687]
 [0.86067298 0.86133018 0.86409043 0.87118822]
 [0.86632492 0.86632492 0.86790221 0.87381703]
 [0.86435331 0.86566772 0.86711356 0.87276551]
 [0.86422187 0.86435331 0.86474763 0.87118822]
 [0.86027865 0.86159306 0.86027865 0.87039958]
 [0.86474763 0.86698212 0.86685068 0.87329127]
 

In [46]:
np.mean(accuracies, axis = 0) 

array([0.86424816, 0.86567298, 0.86628812, 0.87209253])

In [47]:
np.std(accuracies, axis = 0)

array([0.00294896, 0.00285562, 0.00276063, 0.00279578])

In [67]:
y_test_prob[:5,:]

array([[0.95338028, 0.04661972],
       [0.33857172, 0.66142828],
       [0.0449217 , 0.9550783 ],
       [0.56199003, 0.43800997],
       [0.84928133, 0.15071867]])

In [62]:
from sklearn.linear_model import LogisticRegression
train_distances = svm.decision_function(X_train_scaled)
model = LogisticRegression(max_iter=500)
model.fit(train_distances.reshape(-1, 1), y_train)
tmp = model.predict_proba(distances.reshape(-1, 1))

In [64]:
tmp[:5,:]

array([[0.97922567, 0.02077433],
       [0.33148537, 0.66851463],
       [0.03251375, 0.96748625],
       [0.59793167, 0.40206833],
       [0.92415085, 0.07584915]])

In [70]:
tmp1 = np.where(tmp[:,1] > 0.5, 1, 0)
accuracy_score(y_test, tmp1)

0.8715825446898002